## Instracart Market EDA - (a)

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
%matplotlib inline

### Data overview

In [2]:
os.chdir('C:/Users/Bangda/Desktop/kaggle/instacart-market')
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
products = pd.read_csv('products.csv')
orders = pd.read_csv('orders.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')
order_products_train = pd.read_csv('order_products__train.csv')

In [3]:
aisles.shape, departments.shape, products.shape, orders.shape, order_products_prior.shape, order_products_train.shape

((134, 2), (21, 2), (49688, 4), (3421083, 7), (32434489, 4), (1384617, 4))

In [4]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [5]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [6]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [7]:
orders.head() # order_dow is the day of a week

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [8]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [9]:
order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [10]:
test = orders.loc[orders.eval_set == 'test']
train = orders.loc[orders.eval_set == 'train']
prior = orders.loc[orders.eval_set == 'prior']
test.shape, train.shape, prior.shape

((75000, 7), (131209, 7), (3214874, 7))

In [11]:
test.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


In [12]:
# reordered days
orders.days_since_prior_order.value_counts()

30.0    369323
7.0     320608
6.0     240013
4.0     221696
3.0     217005
5.0     214503
2.0     193206
8.0     181717
1.0     145247
9.0     118188
14.0    100230
10.0     95186
13.0     83214
11.0     80970
12.0     76146
0.0      67755
15.0     66579
16.0     46941
21.0     45470
17.0     39245
20.0     38527
18.0     35881
19.0     34384
22.0     32012
28.0     26777
23.0     23885
27.0     22013
24.0     20712
25.0     19234
29.0     19191
26.0     19016
Name: days_since_prior_order, dtype: int64

In [13]:
orders.days_since_prior_order.isnull().sum()

206209

In [14]:
# order_id is the identifier, order_number is quantity
orders.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


#### Append user_id to order_products_prior/table

In [15]:
# number of different product purchased for each user
order_products_prior = pd.merge(order_products_prior, prior[['order_id', 'user_id']], 
                                on = 'order_id', how = 'left')
order_products_train = pd.merge(order_products_train, train[['order_id', 'user_id']],
                                on = 'order_id', how = 'left')

In [16]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered,user_id
0,2,33120,1,1,202279
1,2,28985,2,1,202279
2,2,9327,3,0,202279
3,2,45918,4,1,202279
4,2,30035,5,0,202279


### Interpret prior set, train set and test set

In [17]:
# interpret prior, train, test: take user_id == 1 as example
train.loc[train.user_id == 1]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,train,11,4,8,14.0


In [18]:
prior.loc[prior.user_id == 1]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [19]:
test.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


In [20]:
# prior purchase record (product) for user_id 1
order_products_prior.loc[order_products_prior.user_id == 1].product_id.value_counts()

196      10
12427    10
10258     9
25133     8
46149     3
13032     3
13176     2
49235     2
26088     2
26405     2
17122     1
10326     1
39657     1
38928     1
35951     1
14084     1
30450     1
41787     1
Name: product_id, dtype: int64

In [21]:
# "next" purchase record of user_id 1. we need to predict this from the prior
order_products_train.loc[order_products_train.user_id == 1]

,order_id,product_id,add_to_cart_order,reordered,user_id
484420,1187899,196,1,1,1
484421,1187899,25133,2,1,1
484422,1187899,38928,3,1,1
484423,1187899,26405,4,1,1
484424,1187899,39657,5,1,1
484425,1187899,10258,6,1,1
484426,1187899,13032,7,1,1
484427,1187899,26088,8,1,1
484428,1187899,27845,9,0,1
484429,1187899,49235,10,1,1


In [22]:
# products purchased again by user_id 1
buy_prev = order_products_train.loc[order_products_train.user_id == 1].product_id.values
buy_next = order_products_prior.loc[order_products_prior.user_id == 1].product_id.values
print("user_id 1: prior purchased items:\n{}".format(buy_next))
print("user_id 1: item purchased again (in train):\n{}".format(np.intersect1d(buy_prev, buy_next)))

user_id 1: prior purchased items:
[  196 12427 10258 25133 10326 17122 41787 13176   196 12427 10258 25133
 30450   196 10258 12427 25133 13032   196 12427 10258 25133 26405 49235
 46149 25133   196 10258 12427   196 10258 12427 13176 26088 13032   196
 14084 12427 26088 26405   196 46149 39657 38928 25133 10258 35951 13032
 12427 12427   196 10258 25133 46149 49235   196 12427 10258 25133]
user_id 1: item purchased again (in train):
[  196 10258 13032 25133 26088 26405 38928 39657 46149 49235]


### Re-format the table as: user_id | prior_purchased_items

In [23]:
# start from table: 
# in this table, we can view all distinct product purchased by each user_id
# .. user_id .. | .. product_id ..
# what we want is table like this: user_id | [product_id list]
# order_products_prior_distinct_userid = order_products_prior.groupby(['user_id'])['product_id'].apply(list)
# buy_prev_userid_1 = order_products_prior_distinct_userid[order_products_prior_distinct_userid.index == 1]
# buy_prev_userid_1.values

In [24]:
# prior_userid_product_lst = pd.DataFrame(np.stack((order_products_prior_distinct_userid.index, 
#                                                   order_products_prior_distinct_userid), axis = 1),
#                                        columns = ['user_id', 'product_id_list_prior'])
# prior_userid_product_lst['product_id_list_prior_count'] = prior_userid_product_lst['product_id_list_prior'].apply(lambda x: len(x))
# prior_userid_product_lst.head()

In [25]:
# order_products_train_distinct_userid = order_products_train.groupby(['user_id'])['product_id'].apply(list)
# train_userid_product_lst = pd.DataFrame(np.stack((order_products_train_distinct_userid.index, 
#                                                   order_products_train_distinct_userid), axis = 1),
#                                         columns = ['user_id', 'product_id_list_train'])
# train_userid_product_lst['product_id_list_train_count'] = train_userid_product_lst['product_id_list_train'].apply(lambda x: len(x))
# train_userid_product_lst.head()

### Reordered = 0 removed

In [26]:
order_products_prior_reordered = order_products_prior.loc[order_products_prior.reordered != 0]
order_products_train_reordered = order_products_train.loc[order_products_train.reordered != 0]
order_products_prior.shape, order_products_prior_reordered.shape, order_products_train.shape, order_products_train_reordered.shape

((32434489, 5), (19126536, 5), (1384617, 5), (828824, 5))

In [27]:
order_products_prior_distinct_userid_reordered = order_products_prior_reordered.groupby(['user_id'])['product_id'].apply(list)
prior_userid_product_reordered_lst = pd.DataFrame(np.stack((order_products_prior_distinct_userid_reordered.index, 
                                                            order_products_prior_distinct_userid_reordered), axis = 1),
                                                  columns = ['user_id', 'product_id_list_prior'])
prior_userid_product_reordered_lst['product_id_list_train_count'] = prior_userid_product_reordered_lst['product_id_list_prior'].apply(lambda x: len(x))
prior_userid_product_reordered_lst.head()

,user_id,product_id_list_prior,product_id_list_train_count
0,1,"[196, 12427, 10258, 25133, 13176, 196, 12427, ...",41
1,2,"[32792, 32792, 24852, 34688, 48110, 47209, 320...",93
2,3,"[21903, 17668, 32402, 39190, 47766, 21903, 439...",55
3,4,[35469],1
4,5,"[24535, 43693, 40706, 21413, 13988, 8518, 2660...",14


In [28]:
order_products_train_distinct_userid_reordered = order_products_train_reordered.groupby(['user_id'])['product_id'].apply(list)
train_userid_product_reordered_lst = pd.DataFrame(np.stack((order_products_train_distinct_userid_reordered.index, 
                                                            order_products_train_distinct_userid_reordered), axis = 1),
                                                  columns = ['user_id', 'product_id_list_train'])
train_userid_product_reordered_lst['product_id_list_train_count'] = train_userid_product_reordered_lst['product_id_list_train'].apply(lambda x: len(x))
train_userid_product_reordered_lst.head()

,user_id,product_id_list_train,product_id_list_train_count
0,1,"[196, 25133, 38928, 26405, 39657, 10258, 13032...",10
1,2,"[22963, 7963, 16589, 32792, 41787, 22825, 2485...",12
2,5,"[15349, 21413, 40706, 21616]",4
3,7,"[47272, 37999, 13198, 43967, 40852, 17638, 298...",8
4,8,"[15937, 23165, 21903, 41540]",4


### ...started at 9/7/2017

In [29]:
# number of users: user numbers in train and test are consistent with size; user_id in prior is not distinct
prior['user_id'].unique().size, train['user_id'].unique().size, test['user_id'].unique().size

(206209, 131209, 75000)

In [30]:
order_products_train.shape, order_products_prior.shape

((1384617, 5), (32434489, 5))

In [31]:
# number of users in order_products_train and order_products_prior are consistent with in prior, train
order_products_train['user_id'].unique().size, order_products_prior['user_id'].unique().size 

(131209, 206209)

In [32]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered,user_id
0,2,33120,1,1,202279
1,2,28985,2,1,202279
2,2,9327,3,0,202279
3,2,45918,4,1,202279
4,2,30035,5,0,202279


In [33]:
order_products_prior.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 5 columns):
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
user_id              int64
dtypes: int64(5)
memory usage: 1.4 GB


In [34]:
# remove reoredered == 0 in order_products_prior
order_products_prior_reordered = order_products_prior.loc[order_products_prior['reordered'] > 0]
order_products_prior_reordered.shape

(19126536, 5)

In [35]:
# try to format like this: user_id (distinct) | product_id (distinct) | count
user_product_count = order_products_prior_reordered.groupby(['user_id', 'product_id'])[['product_id']].count()
user_product_count.columns = ['product_count']
user_product_count.shape

(5325258, 1)

In [36]:
user_product_count = user_product_count.reset_index()
user_product_count.head()

,user_id,product_id,product_count
0,1,196,9
1,1,10258,8
2,1,12427,9
3,1,13032,2
4,1,13176,1


In [37]:
print('Number of users in prior (reordered): {}'.format(user_product_count.user_id.unique().shape[0]))
print('Number of products in prior (reordered): {}'.format(user_product_count.product_id.unique().shape[0]))

Number of users in prior (reordered): 203164
Number of products in prior (reordered): 45305


In [38]:
# Try to form a matrix where each row represents one user and each column represents one product
# MEMORY ERROR
# user_product_matrix = np.zeros((user_product_count.user_id.unique().shape[0], user_product_count.product_id.unique().shape[0]))
# for row in user_product_count.itertuples():
#     user_product_matrix[row[1] - 1, row[2] - 1] = row[3]

In [39]:
user_product_count_subset = user_product_count.iloc[:20000, :]
print('Number of users in prior subset (reordered): {}'.format(user_product_count_subset.user_id.unique().shape[0]))
print('Number of products in prior subset (reordered): {}'.format(user_product_count_subset.product_id.unique().shape[0]))

Number of users in prior subset (reordered): 820
Number of products in prior subset (reordered): 7132


In [40]:
user_product_subset_matrix = np.zeros((user_product_count_subset.user_id.unique().shape[0], user_product_count_subset.product_id.unique().shape[0]))
user_product_subset_matrix.shape
#for row in user_product_count_subset.itertuples():
#    user_product_subset_matrix[row[1] - 1, row[2] - 1] = row[3]

(820, 7132)

In [41]:
user_product_raw_id = user_product_count_subset['product_id'].value_counts().index.values
user_product_raw_id = pd.DataFrame(user_product_raw_id)
user_product_raw_id['new_product_id'] = np.arange(7132) # user_product_subset_matrix.shape[1]
user_product_raw_id.columns = ['raw_product_id', 'new_product_id']
user_product_raw_id.head()

,raw_product_id,new_product_id
0,24852,0
1,13176,1
2,21137,2
3,21903,3
4,47209,4
